## Modeling

This notebook contains code for

1. Training of P-Model

2. Training of PF-Model

In order to keep this notebook clearly readable, some functions are outsourced in utils/

____

### Imports

In [ ]:
import pandas as pd
import pickle

from sklearn.preprocessing import StandardScaler

from keras.models import Sequential
from keras.layers import GRU, Dense, Dropout
from keras.models import load_model, plot_model
from keras.regularizers import l1, l2, l1_l2
from keras.optimizers import Adam

from livelossplot import PlotLossesKeras

import plotly.graph_objs as go

____

#### 1. P-Model

Load data

In [ ]:
from utils.data_utils import data_loader
from utils.utils import train_test_val_data
import config

# load data
data = data_loader(config.columns_P)
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)
df_scaled = pd.DataFrame(scaled_data, columns=data.columns)


X_train, X_val, X_test, y_train, y_val, y_test = train_test_val_data(df_scaled, 
                                                                     len(data.index.unique()), 
                                                                     1)

In [ ]:
num_gru_layers = 2
units_0, units_1, units_2, units_output = 96, 512, 128, 96
dropout_0, dropout_1, dropout_2 = 0.4, 0.2, 0
batch_size = 64
learning_rate = 0.0014318
l1 = 1.3801e-06
l2 = 9.534e-06
epochs=10

In [ ]:
model_final = Sequential()
# Input Layer
model_final.add(GRU(units=units_0, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model_final.add(Dropout(dropout_0)) 
# Hidden Layers
model_final.add(GRU(units=units_1,return_sequences=True))
model_final.add(Dropout(dropout_1))
model_final.add(GRU(units=units_2,return_sequences=False))
model_final.add(Dropout(dropout_2))
# Output Layer
model_final.add(Dense(units_output, activation='linear', 
                kernel_regularizer=l1_l2(l1=l1, l2=l2)
                ))

# Compile
model_final.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error', metrics=['mean_absolute_error'])

# Trainieren des Modells
history = model_final.fit(
    X_train, y_train,                   
    epochs=epochs,                          
    batch_size=batch_size,                   
    validation_data=(X_val, y_val),     
    callbacks=[PlotLossesKeras()],
    use_multiprocessing=True,          
    workers=4,                          
    verbose=1,                          
)

with open('data/models/history/P_optimized_final_history.pkl', 'wb') as f:
    pickle.dump(history, f)

#### 2. PF-Model

Load data

In [ ]:
# load data
data = data_loader(config.columns_PF)
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)
df_scaled = pd.DataFrame(scaled_data, columns=data.columns)


X_train, X_val, X_test, y_train, y_val, y_test = train_test_val_data(df_scaled, 
                                                                     len(data.index.unique()), 
                                                                     1)

In [ ]:
units_0, units_1, units_2 = 256, 192, 192
dropout_0, dropout_1, dropout_2 = 0.1, 0.0, 0.4
batch_size = 64
units_output = 96
epochs=34
learning_rate=0.003979

model_final = Sequential()
# Input Layer
model_final.add(GRU(units=units_0, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model_final.add(Dropout(dropout_0)) 
# Hidden Layers
model_final.add(GRU(units=units_1, return_sequences=True))
model_final.add(Dropout(dropout_1)) 
model_final.add(GRU(units=units_2,return_sequences=False))
model_final.add(Dropout(dropout_2))
# Output Layer
model_final.add(Dense(units_output, activation='linear'))

# Compile
model_final.compile(optimizer="adam", loss='mean_squared_error', metrics=['mean_absolute_error'])

# Trainieren des Modells
history = model_final.fit(
    X_train, y_train,                   
    epochs=epochs,                          
    batch_size=batch_size,                      
                                       
    validation_data=(X_val, y_val),     
    callbacks=[PlotLossesKeras()],
    use_multiprocessing=True,          
    workers=4,                          
    verbose=1,                          
)

with open('data/models/history/PF_optimized_final_history.pkl', 'wb') as f:
    pickle.dump(history, f)

model_final.save('data/models/GRU_PF_final.h5')  # Speichert das Modell im HDF5-Format